In [1]:
## Loading Config Files

import os 
import json
import sys
sys.path.append("..")

with open('../config/runner_config.json') as json_file:
    json_data = json.load(json_file)


In [3]:
## Data Aggreagator Run
from deployment.Data_Retreiver import Data_Retreiver
from deployment.Data_Aggregator import Aggregator

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

sql_table = json_data["sql_aggregate"]
sql_table_raw = json_data["sql_raw_energy"]


# Create Threads for All the devices
aggre = Aggregator(devices,sql_table,sql_table_raw,sql_addr,sql_port, sql_user, sql_pw, sql_db)

#Do step and get the final output that would be printed to the log
aggre.do_step()
print(aggre.getLatest())

Aggregator: 2020-06-25 16:21:27.140458:    Starting Routine
------------For Time 2020-06-24 19:30:00 
Missing Value found for Row [nursery1_lights-Nursery_1B_CPE_No_3-LED1_P; nursery1_lights-Nursery_1C_CPE_No_5-LED1_P; nursery1_lights-Nursery_1C_CPE_No_5-LED2_P; nursery1_lights-Nursery_1C_CPE_No_5-LED3_P; nursery1_sockets-Nursery_AC_Socket_1C-vRELAY1_LVL; nursery2_lights-Nursery_2B_CPE_No_9-LED1_P; nursery2_lights-Nursery_2B_CPE_No_9-LED2_P; nursery2_lights-Nursery_2B_CPE_No_9-LED3_P; nursery2_lights-Nursery_2C_CPE_No_12-LED2_P; nursery2_lights-Nursery_2C_CPE_No_12-LED3_P; nursery2_sockets-Nursery_AC_Socket_2A_No_1-vRELAY1_LVL; nursery2_sockets-Nursery_AC_Socket_2A_No_2-vRELAY1_LVL; nursery2_sockets-Nursery_AC_Socket_2C-vRELAY1_LVL; playground_lights-Playground_No_2-LED3_P; playground_sockets-Playground_AC_Socket_No_2-vRELAY1_LVL; streetlights-Streetlight_No_1-LED1_P; streetlights-Streetlight_No_1-LED2_P; streetlights-Streetlight_No_1-LED3_P; ] Usually IndexError: index 0 is out of bou

In [4]:
## Conctrol Enactor 
from deployment.Control_Enactor import Enactor


devices = json_data["devices"]

sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

# Run Examples
print("Retreive - Set Points")
print(enact.retreive_previous_plan_DC(list(devices.keys())[1]))
print(enact.retreive_previous_plan_AC(list(devices.keys())[9]))
print("Enact Set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Same set points")
print(enact.enact_light_plan(list(devices.keys())[1], 2100, 2100))
print(enact.enact_socket_plan(list(devices.keys())[9], 1000000))
print("Enact Diffrent set points")
print(enact.enact_light_plan(list(devices.keys())[1], 4320, 4320))
print(enact.enact_socket_plan(list(devices.keys())[9], 1200000))


Retreive - Set Points
(4320, 4320)
1200000
Enact Set points
True
True
Enact Same set points
False
False
Enact Diffrent set points
True
True


In [5]:
#Data retreiver Run
import datetime
import pandas as pd 
from deployment.Data_Retreiver import Data_Retreiver
devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

print("--Retreive filled data--\n",data.get_unsent_logs(datetime.datetime.now()))
print("--Retreive filled data--\n",data.retreive_filled_aggre(datetime.datetime.now() - datetime.timedelta(days=20), 2))

dev_light = "Nursery_1A_CPE_No_2"
dev_socket = "Playground_AC_socket_No_1"
group = ["Nursery_1A_CPE_No_1", "Nursery_1A_CPE_No_2", "Nursery_1B_CPE_No_3", "Nursery_1B_CPE_No_4",
         "Nursery_1C_CPE_No_5", "Nursery_1C_CPE_No_6"]
hour = 6

pd.set_option('display.max_columns', 30)
dt_ts = datetime.datetime.now()
print("--Forecast--\n", data.retreive_latest_forecast(dt_ts).head(5))
print("--Raw Sys--\n", data.retreive_latest_raw_system_snapshot(dt_ts).head(5))
print("--Priority--\n", data.retreive_latest_priority(dt_ts).head(5))
print("--Group Energy--\n", data.get_total_energy_for_group(group, dt_ts))
print("--Aggregated Values --\n", data.retreive_aggregared_values(dt_ts))
print("--AVG for Hour Values --\n", data.retreive_average_vals_for_hour(dt_ts, hour,8))
print("--Average P Light--\n", data.retreive_average_P_lights(dev_light, dt_ts))
print("--AC Session--\n", data.retreive_AC_Session(dev_socket, dt_ts))
print("--AC Energy--\n", data.retreive_AC_Energy(dev_socket, dt_ts))
print("--Light Session--\n", data.retreive_Light_Session(dev_light, dt_ts))
print("--Light Energy--\n", data.retreive_Light_Energy(dev_light, dt_ts))
print("--Quota Latest Light--\n", data.get_latest_quota(dev_light, dt_ts))
print("--Quota Latest Socket --\n", data.get_latest_quota(dev_socket, dt_ts))



No Data Available for unsent logs 2020-06-25 16:21:44.096611  at:  2020-06-25 16:21:44.123614
--Retreive filled data--
 None
Value Error probably not data avilable for period:>  2020-06-05 16:15:00   2  Error: Length mismatch: Expected axis has 0 elements, new values have 37 elements
--Retreive filled data--
 None
--Forecast--
    system_load  generated_energy
0   1822.81768        630.866667
--Raw Sys--
                   timestamp                        parameter   value
0 2020-06-25 14:28:16+00:00         VenusGX/Dc/Battery/Power  1098.0
1 2020-06-25 14:28:16+00:00              VenusGX/Dc/Pv/Power  1100.0
2 2020-06-25 14:28:16+00:00           VenusGX/Dc/Battery/Soc   100.0
3 2020-06-25 14:28:16+00:00  VenusGX/Ac/Consumption/L1/Power   155.0
--Priority--
    id                        timestamp  nursery1_lights  nursery1_sockets  \
0   5 2020-06-22 15:51:23.785037+00:00                1                 4   

   nursery2_lights  nursery2_sockets  playground_lights  playground_sockets  

TypeError: object of type 'NoneType' has no len()

In [6]:
from deployment.Battery import Battery

bat = Battery(state_of_charge=94.0, battery_capacity=18.79, max_discharge=40.0, charge_efficiency=0.85)
print(bat)
print("---Storage Capacity---\n", bat.get_storage_capacity_left())
print("---Discharge Capacity---\n", bat.get_discharge_capacity_left())
print("---Discharge Battery---\n", bat.discharge_battery(1.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(0.4))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Discharge Battery---\n", bat.discharge_battery(2.9))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)
print("---Charge Battery---\n", bat.charge_battery(4.4))  # in KWH
print(bat)


SoC: 94.0 Energy_Left: 10.1466
---Storage Capacity---
 1.3263529411764705
---Discharge Capacity---
 10.1466
---Discharge Battery---
 0.0
SoC: 83.88823842469398 Energy_Left: 8.246599999999999
---Charge Battery---
 0.0
SoC: 85.6977115486961 Energy_Left: 8.586599999999999
---Discharge Battery---
 0.0
SoC: 70.26397019691323 Energy_Left: 5.686599999999997
---Discharge Battery---
 0.0
SoC: 54.83022884513037 Energy_Left: 2.786599999999997
---Discharge Battery---
 0.11340000000000305
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Discharge Battery---
 2.9
SoC: 40.0 Energy_Left: 0.0
---Charge Battery---
 0.0
SoC: 59.90420436402342 Energy_Left: 3.7400000000000007
---Charge Battery---
 0.0
SoC: 79.80840872804684 Energy_Left: 7.480000000000001
---Charge Battery---
 0.0
SoC: 99.71261309207026 Energy_Left: 11.22
---Charge Battery---
 4.336470588235296
SoC: 100.0 Energy_Left: 11.274
---Charge Battery---
 4.4
SoC: 100.0 Energy_Left: 11.274


In [9]:
# Controller Run

#Data retreiver Run
import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Control_Enactor import Enactor
from deployment.Controller import Controller

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

## Conctrol Enactor 
sftp_location = json_data["sftp_location"]
sftp_port = json_data["sftp_port"]
sftp_username = json_data["sftp_username"]
sftp_password = json_data["sftp_password"]
sftp_key_location = json_data["sftp_key_location"]
sftp_directory = json_data["sftp_directory"]

#Global for Unit Test
enact = Enactor(devices, sftp_location, sftp_port, sftp_username, sftp_password, sftp_key_location, sftp_directory)

#Controller Init
ts_step = datetime.datetime.now()  # - datetime.timedelta(hours=3)

allocation = json_data["allocation"]
if 'System_Data' in allocation:
    del allocation['System_Data']

control = Controller(data, enact, allocation, 4)
print(control.getLatest())

control.do_step(ts_step)
print(control.getLatest())

Controller : Initilised
Controller : Latest Data From: 2020-06-25 16:25:01.581688
-------Energy State--------
Generated energy: 1414.8666666666668
System Load: 1580.1287022928088
Battery SoC: 100.0
Remaining Energy: 10912.712223915296
{1: 'nursery1_lights', 4: 'nursery1_sockets', 2: 'nursery2_lights', 5: 'nursery2_sockets', 3: 'playground_lights', 6: 'playground_sockets'}
------------------------------------------------
For Device: nursery1_lights with energy avialable: 10912.712223915296
Decisions: {'state': 'Unconstrained', 'energy_est_used_total': 122.84671149425289, 'constraining_factor': 1.0, 'device_const': {'Nursery_1A_CPE_No_1': (4320, 4320, False), 'Nursery_1A_CPE_No_2': (4320, 4320, False), 'Nursery_1B_CPE_No_3': (4320, 4320, False), 'Nursery_1B_CPE_No_4': (4320, 4320, False), 'Nursery_1C_CPE_No_5': (4320, 4320, False), 'Nursery_1C_CPE_No_6': (4320, 4320, False)}, 'timestamp': '2020-06-25 16:25:01.581688'}
Remaining: 10765.296170122192
----------------------------------------

In [8]:
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.latest_forecast)

Running Forecasting --  2020-06-25 16:24:46.939329  --
Forecaster : 2020-06-25 16:24:53.118114 : [{"system_load":207.0022541029,"generated_energy":0.0,"consumed_energy":296.0}]


parameter,nursery_1a_cpe_no_1,nursery_1b_cpe_no_3,nursery_1b_cpe_no_4,nursery_1c_cpe_no_5,nursery_1c_cpe_no_6,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2c_cpe_no_12,nursery_ac_socket_1a_no_1,nursery_ac_socket_1a_no_2,nursery_ac_socket_1b,nursery_ac_socket_1c,nursery_ac_socket_2a_no_1,nursery_ac_socket_2a_no_2,...,streetlight_no_1,streetlight_no_2,streetlight_no_3,battery_soc,charged_energy,consumed_energy,generated_energy,timestamp,system_load,victron_venusgx,playground_no_1,playground_no_4,nursery_1a_cpe_no_2,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11
timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-06-25 14:00:00,0.0000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,2.105821,0.0,0.0,0.000000,0.000000,0.00000,6.000000,...,9.280000,0.159765,5.535478,98.791469,-255.000000,255.0,0.000000,2020-06-25 14:00:00,168.395436,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 15:00:00,3.1830,2.851000,0.0,6.179000,0.0,5.006000,0.0,2.626000,7.238000,3.0,0.0,0.000000,0.000000,0.00000,1.000000,...,8.887000,0.159765,7.265136,97.388626,-296.000000,296.0,0.000000,2020-06-25 15:00:00,207.002254,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 16:00:00,7.9310,2.120333,0.0,6.931000,0.0,4.679000,0.0,7.160000,3.391000,3.0,0.0,0.000000,0.000000,0.00000,3.000000,...,7.318000,0.159765,10.145000,96.056872,-281.000000,281.0,0.000000,2020-06-25 16:00:00,185.893402,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 17:00:00,2.1849,2.120333,0.0,4.329000,0.0,1.984897,0.0,4.195000,6.435000,1.5,0.0,2.241379,0.000000,4.00000,2.000000,...,7.860000,0.159765,4.739304,96.280987,55.633333,172.3,227.933333,2020-06-25 17:00:00,118.701313,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 18:00:00,3.1410,7.753000,0.0,2.277345,0.0,4.739000,0.0,1.796069,2.105821,3.0,0.0,0.000000,0.793103,1.62069,1.310345,...,2.104966,0.159765,4.608136,95.006106,-269.000000,269.0,0.000000,2020-06-25 18:00:00,193.565610,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 19:00:00,7.2540,5.951000,0.0,2.813000,0.0,4.929000,0.0,3.240000,5.136000,3.0,0.0,0.000000,0.000000,5.00000,0.000000,...,6.868000,0.159765,8.029000,93.745442,-266.000000,266.0,0.000000,2020-06-25 19:00:00,187.886235,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 20:00:00,5.7260,7.723000,0.0,5.897000,0.0,4.572000,0.0,2.569000,5.754000,3.0,0.0,0.000000,0.000000,3.00000,0.000000,...,10.461000,0.159765,9.390000,92.513215,-260.000000,260.0,0.000000,2020-06-25 20:00:00,173.655235,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 21:00:00,2.1849,2.120333,0.0,5.012000,0.0,4.881000,0.0,4.144000,2.093000,1.5,0.0,2.241379,0.000000,0.00000,0.000000,...,1.646000,0.159765,2.474441,92.737330,55.633333,172.3,227.933333,2020-06-25 21:00:00,136.613768,0.0,0.0,0.0,0.0,0.0,0.0
2020-06-25 22:00:00,5.3220,4.281000,0.0,6.793500,0.0,4.909000,0.0,5.798000,2.896000,1.5,0.0,0.000000,0.000000,0.00000,0.000000,...,1.518500,0.159765,2.474441,91.846335,-188.000000,188.0,0.000000,2020-06-25 22:00:00,143.997358,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
##Forecaster V2 - ML and PV Models
## Forecaster V1 - Simple not ML

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Battery import Battery
from deployment.Forecaster_v2 import Forecaster

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

# Init Data Retreiver
data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)
forecast = Forecaster(Battery, data, devices)


curr_ts = datetime.datetime.now()
print("Running Forecasting -- ", curr_ts, " --")
forecast_period = 48
forecast.do_step(forecast_period, curr_ts)
print(forecast.getLatest())
#Latest Values
display(forecast.full_df)

Running Forecasting --  2020-06-23 13:02:08.331570  --
Doing forecast for Dev : consumed_energy
Doing forecast for Dev : system_load
Doing forecast for Dev : streetlight_no_1
Doing forecast for Dev : streetlight_no_2
Doing forecast for Dev : streetlight_no_3
Doing forecast for Dev : playground_no_1
Doing forecast for Dev : playground_no_2
Doing forecast for Dev : playground_no_3
Doing forecast for Dev : playground_no_4
Doing forecast for Dev : playground_no_5
Doing forecast for Dev : playground_ac_socket_no_1
Doing forecast for Dev : playground_ac_socket_no_2
Doing forecast for Dev : nursery_ac_socket_1a_no_1
Doing forecast for Dev : nursery_ac_socket_1a_no_2
Doing forecast for Dev : nursery_ac_socket_1b
Doing forecast for Dev : nursery_ac_socket_1c
Doing forecast for Dev : nursery_1a_cpe_no_1
Doing forecast for Dev : nursery_1a_cpe_no_2
Doing forecast for Dev : nursery_1b_cpe_no_3
Doing forecast for Dev : nursery_1b_cpe_no_4
Doing forecast for Dev : nursery_1c_cpe_no_5
Doing forecast 

,consumed_energy,timestamp,system_load,streetlight_no_1,streetlight_no_2,streetlight_no_3,playground_no_1,playground_no_2,playground_no_3,playground_no_4,...,nursery_ac_socket_2c,nursery_2a_cpe_no_7,nursery_2a_cpe_no_8,nursery_2b_cpe_no_9,nursery_2b_cpe_no_10,nursery_2c_cpe_no_11,nursery_2c_cpe_no_12,generated_energy,charged_energy,battery_soc
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-06-23 14:00:00,10.570274,2020-06-23 14:00:00,123.005550,3.740689e-16,2.854787e-30,-1.724892e-15,0.0,0.000015,-8.946975e-38,0.0,...,5.554236e-06,3.856813e-16,0.0,2.154610e-17,0.0,0.0,-1.834440e-39,265.239115,254.668841,1.02592
2020-06-23 15:00:00,220.859129,2020-06-23 15:00:00,120.886324,2.822000e+00,9.459227e-01,2.872000e+00,0.0,0.000024,2.813497e+00,0.0,...,9.608059e-06,3.987500e-01,0.0,1.837500e-01,0.0,0.0,7.370002e-01,55.524846,-165.334283,40
2020-06-23 16:00:00,308.093937,2020-06-23 16:00:00,118.384877,2.723928e+00,8.805364e-01,3.383275e+00,0.0,0.000024,3.193675e+00,0.0,...,5.546024e-06,1.595089e+00,0.0,7.552084e-01,0.0,0.0,1.437150e+00,0.160249,-307.933688,40
2020-06-23 17:00:00,331.793063,2020-06-23 17:00:00,115.730514,2.697000e+00,8.107909e-01,3.087850e+00,0.0,0.000026,3.085496e+00,0.0,...,3.201311e-06,1.596250e+00,0.0,1.017917e+00,0.0,0.0,9.581002e-01,0.000000,-331.793063,40
2020-06-23 18:00:00,348.316739,2020-06-23 18:00:00,113.159406,2.663857e+00,7.410454e-01,2.724250e+00,0.0,0.000028,2.952354e+00,0.0,...,1.847881e-06,1.597678e+00,0.0,1.341250e+00,0.0,0.0,3.685001e-01,0.000000,-348.316739,40
2020-06-23 19:00:00,365.930658,2020-06-23 19:00:00,110.615531,2.630714e+00,6.713000e-01,2.524000e+00,0.0,0.000030,2.819211e+00,0.0,...,1.066645e-06,1.599107e+00,0.0,1.664584e+00,0.0,0.0,-2.775559e-17,0.000000,-365.930658,40
2020-06-23 20:00:00,388.068558,2020-06-23 20:00:00,107.743902,2.512750e+00,6.015545e-01,2.504000e+00,0.0,0.000024,2.737497e+00,0.0,...,6.156957e-07,1.584000e+00,0.0,1.987917e+00,0.0,0.0,1.905250e+00,0.000000,-388.068558,40
2020-06-23 21:00:00,395.026464,2020-06-23 21:00:00,105.080943,2.230917e+00,5.318091e-01,2.668167e+00,0.0,0.000033,2.681830e+00,0.0,...,3.553957e-07,1.540750e+00,0.0,2.098500e+00,0.0,0.0,1.089750e+00,0.000000,-395.026464,40
2020-06-23 22:00:00,300.372147,2020-06-23 22:00:00,101.808207,2.330083e+00,4.620636e-01,2.989833e+00,0.0,0.000035,2.080164e+00,0.0,...,2.051437e-07,1.588250e+00,0.0,1.183500e+00,0.0,0.0,1.837250e+00,0.000000,-300.372147,40


In [10]:
##Remote Logger Example

import datetime
import pandas as pd 

from deployment.Data_Retreiver import Data_Retreiver
from deployment.Remote_Logger import RemoteLogger

devices = json_data["devices"]

# Data Retreiver
sql_user = json_data["sql_user"]
sql_pw = json_data["sql_pw"]
sql_db = json_data["sql_db"]
sql_addr = json_data["sql_addr"]
sql_port = json_data["sql_port"]

mqtt_address = json_data["mqtt_address"]
mqtt_port = json_data["mqtt_port"]
mqtt_username = json_data["mqtt_username"]
mqtt_password = json_data["mqtt_password"]

data = Data_Retreiver(devices, sql_user, sql_pw, sql_addr, sql_port, sql_db)

rl = RemoteLogger(data, mqtt_address, mqtt_port, mqtt_username, mqtt_password)

rl.do_step()
print(rl.getLatest())

Remote Logger: 2020-06-25 16:25:49.843317 : Publishing message with Nulls:['hist_playground_no_1', 'hist_playground_no_4', 'hist_nursery_1a_cpe_no_2', 'hist_nursery_2b_cpe_no_10', 'hist_nursery_2c_cpe_no_11', 'quota2_nursery_ac_socket_1a_no_1', 'quota2_nursery_ac_socket_1a_no_2', 'quota2_nursery_ac_socket_1b', 'quota2_nursery_ac_socket_1c', 'quota2_nursery_ac_socket_2a_no_1', 'quota2_nursery_ac_socket_2a_no_2', 'quota2_nursery_ac_socket_2b', 'quota2_nursery_ac_socket_2c', 'quota2_playground_ac_socket_no_2']  with return:(0, 1)
